<!-- PS-S5.E01 -->

<div style="font-family: 'Poppins'; font-weight: bold; letter-spacing: 0px; color: #FFFFFF; font-size: 500%; text-align: center; padding: 15px; background: #0A0F29; border: 8px solid #00FFFF; border-radius: 15px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5);">
    LLM : EDA and code generation <br>
</div>

- simple example on how to use LLM for:
    - generating EDA summaries
    - generating code for initial baseline

- Next steps:
    - Include agents in the workflow
    - Pass along a summary of the competition instructions

# <div style="background-color:#0A0F29; font-family:'Poppins', bold; color:#E0F7FA; font-size:140%; text-align:center; border: 2px solid #00FFFF; border-radius:15px; padding: 15px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5); font-weight: bold; letter-spacing: 1px; text-transform: uppercase;">Generate an EDA summary</div>

In [1]:
!pip install openai==1.58.1 langchain-core langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [2]:
BASE_LLM = 'gpt-4o-2024-05-13'
ADVANCED_LLM = 'o1-preview'
SELECTED_LLM = BASE_LLM
TEMPERATURE = 0
MAX_TOKENS=3000

In [3]:
# Standard Library Imports
import os
import datetime
import json

## LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from IPython.display import display, Markdown

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
OPENAI_API_KEY = user_secrets.get_secret("openai_key")

In [5]:
context = """ 


"general": {
"num_rows": 230130,
"num_columns": 6,
"num_missing_values": "8871",
"percent_missing_values": 0.6424629557206796
},
"data_types": {
"date": "object",
"country": "object",
"store": "object",
"product": "object",
"num_sold": "float64"
},
"missing_values": {
"date": {
"missing_count": 0,
"percent_missing": 0.0
},
"country": {
"missing_count": 0,
"percent_missing": 0.0
},
"store": {
"missing_count": 0,
"percent_missing": 0.0
},
"product": {
"missing_count": 0,
"percent_missing": 0.0
},
"num_sold": {
"missing_count": 8871,
"percent_missing": 3.8547777343240774
}
},
"numerical_summary": {
"count": {},
"mean": {},
"std": {},
"min": {},
"25%": {},
"50%": {},
"75%": {},
"max": {}
},
"categorical_summary": {
"date": {
"unique_counts": 2557
},
"country": {
"unique_counts": 6
},
"store": {
"unique_counts": 3
},
"product": {
"unique_counts": 5
}
},
"skewness_kurtosis": {
"num_sold": {
"skewness": 1.415373452498392,
"kurtosis": 2.6123350629213618
}
},
"correlations": {
"num_sold": {
"num_sold": 1.0
}
},
"outlier_summary": {
"num_sold": {
"outlier_count": 6630,
"percent_outliers": 2.8809803154738627
}
}
}

"""

In [6]:
template = """


Provide an analysis of the following EDA summary: The target variable is num_sold.
{context}

Add a comment about the missing values in the target variable: num_sold. And the implications if those are missing at random or not. 
Key Insights and Observations


"""

In [7]:
# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

# Prepare parameters for ChatOpenAI
model_params = {
    "model": SELECTED_LLM,
    "api_key": OPENAI_API_KEY
}


display(Markdown(f'**selected model: {SELECTED_LLM}**'))

# Conditionally set temperature if supported
if SELECTED_LLM != ADVANCED_LLM:
    model_params["temperature"] = TEMPERATURE 
    model_params["max_tokens"] = MAX_TOKENS

if SELECTED_LLM == ADVANCED_LLM:
    model_params["max_completion_tokens"] = MAX_COMPLETION_TOKENS

# Initialize the model with the appropriate parameters
model = ChatOpenAI(**model_params)

# Create the processing chain
chain = prompt | model | StrOutputParser()

try:
    # Invoke the chain to get the result
    result = chain.invoke(context)

    # Save both the prompt and the result to a Markdown file
    file_path = '/kaggle/working/output_base_model.md'
    with open(file_path, 'w') as f:
        f.write("# EDA Report\n\n")
        f.write("## Prompt\n")
        f.write(template.format(context=context))
        f.write("\n\n## Response\n")
        f.write(result)

    # Display the result as Markdown in the notebook
    display(Markdown(result))

    display(Markdown(f"**Markdown report saved to: {file_path}**"))

except BadRequestError as e:
    print(f"An error occurred: {e}")

**selected model: gpt-4o-2024-05-13**

### Comment on Missing Values in the Target Variable: `num_sold`

The target variable `num_sold` has 8,871 missing values, which constitutes approximately 3.85% of the total dataset. This is a significant amount of missing data that needs to be addressed before any modeling or analysis can be performed.

#### Implications if Missing at Random (MAR):
If the missing values in `num_sold` are missing at random, it means that the likelihood of a value being missing is related to some of the observed data but not the missing data itself. In this case, the missing values can be handled using imputation techniques such as mean, median, or more sophisticated methods like multiple imputation. The assumption here is that the missing data can be predicted based on the observed data.

#### Implications if Not Missing at Random (NMAR):
If the missing values are not missing at random, it means that the missingness is related to the unobserved data itself. This scenario is more problematic because it introduces bias that cannot be easily corrected. For example, if `num_sold` is missing more frequently for certain products or stores, this could skew the analysis and lead to incorrect conclusions. In such cases, it might be necessary to use more advanced techniques like modeling the missing data mechanism or even collecting more data if possible.

### Key Insights and Observations

1. **General Overview**:
   - The dataset contains 230,130 rows and 6 columns.
   - There are 8,871 missing values, which is about 0.64% of the total data.

2. **Data Types**:
   - The dataset includes both categorical (date, country, store, product) and numerical (num_sold) data types.

3. **Missing Values**:
   - The missing values are entirely in the `num_sold` column, with a missing rate of 3.85%.

4. **Categorical Summary**:
   - The `date` column has 2,557 unique values.
   - The `country` column has 6 unique values.
   - The `store` column has 3 unique values.
   - The `product` column has 5 unique values.

5. **Skewness and Kurtosis**:
   - The `num_sold` variable has a skewness of 1.415, indicating a right-skewed distribution.
   - The kurtosis is 2.612, suggesting a distribution with heavier tails than a normal distribution.

6. **Correlations**:
   - The correlation matrix shows that `num_sold` is perfectly correlated with itself, as expected.

7. **Outliers**:
   - There are 6,630 outliers in the `num_sold` column, which is about 2.88% of the data. This indicates the presence of extreme values that could affect the analysis.

### Recommendations

1. **Handling Missing Values**:
   - Investigate the pattern of missingness to determine if it is MAR or NMAR.
   - Use appropriate imputation techniques if the data is MAR.
   - Consider advanced methods or data collection if the data is NMAR.

2. **Outlier Treatment**:
   - Analyze the outliers to understand their impact on the model.
   - Consider using robust statistical methods or transforming the data to mitigate the effect of outliers.

3. **Further Analysis**:
   - Perform additional exploratory data analysis to understand the relationships between `num_sold` and other variables.
   - Consider feature engineering to create new variables that might help in predicting `num_sold`.

By addressing the missing values and outliers appropriately, the dataset can be prepared for more accurate and reliable modeling and analysis.

**Markdown report saved to: /kaggle/working/output_base_model.md**

# <div style="background-color:#0A0F29; font-family:'Poppins', bold; color:#E0F7FA; font-size:140%; text-align:center; border: 2px solid #00FFFF; border-radius:15px; padding: 15px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5); font-weight: bold; letter-spacing: 1px; text-transform: uppercase;">LLM automated Baseline</div>

In [8]:
TEMPERATURE = 1
MAX_TOKENS=3500
MAX_ITERATIONS = 10

- Used a simplified version of [S5E1 Previous Years Baseline - No Model](https://www.kaggle.com/code/cabaxiom/s5e1-previous-years-baseline-no-model) as template for the LLM (previous best score was 0.12585)

In [9]:
best_model_script = """
# =========================================
# 1. LIBRARIES & CONFIGURATION
# =========================================
import numpy as np
import pandas as pd
import warnings
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# =========================================
# 2. DATA LOADING
# =========================================
# Paths to the datasets
TRAIN_PATH = "/kaggle/input/playground-series-s5e1/train.csv"
TEST_PATH = "/kaggle/input/playground-series-s5e1/test.csv"
GDP_PATH = "/kaggle/input/world-gdpgdp-gdp-per-capita-and-annual-growths/gdp_per_capita.csv"
SUBMISSION_PATH = "/kaggle/input/playground-series-s5e1/sample_submission.csv"

# Load datasets
train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
test_df = pd.read_csv(TEST_PATH, parse_dates=["date"])
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# =========================================
# 3. PREPROCESSING & IMPUTING MISSING VALUES
# =========================================

# Read GDP per capita data
gdp_df = pd.read_csv(GDP_PATH)
years = [str(year) for year in range(2010, 2021)]  # 2010 to 2020 inclusive

# Filter relevant countries and years from GDP data
relevant_countries = train_df["country"].unique()
gdp_filtered = gdp_df[gdp_df["Country Name"].isin(relevant_countries)]
gdp_filtered = gdp_filtered[["Country Name"] + years]
gdp_filtered = gdp_filtered.melt(id_vars=["Country Name"], var_name="year", value_name="gdp_per_capita")
gdp_filtered.rename(columns={"Country Name": "country"}, inplace=True)
gdp_filtered['year'] = gdp_filtered['year'].astype(int)

# Calculate GDP ratios per year
gdp_total = gdp_filtered.groupby('year')['gdp_per_capita'].sum().reset_index().rename(columns={'gdp_per_capita': 'total_gdp'})
gdp_filtered = gdp_filtered.merge(gdp_total, on='year', how='left')
gdp_filtered['gdp_ratio'] = gdp_filtered['gdp_per_capita'] / gdp_filtered['total_gdp']
gdp_filtered.drop(columns=['gdp_per_capita', 'total_gdp'], inplace=True)

# Prepare train data for imputation
train_df['year'] = train_df['date'].dt.year
print(f"Missing values before imputation: {train_df['num_sold'].isna().sum()}")

# Merge train data with GDP ratios
train_df = train_df.merge(gdp_filtered, on=['country', 'year'], how='left')

# Impute missing values
# Calculate average num_sold per (date, store, product) in Norway
norway_avg = train_df[train_df['country'] == 'Norway'].groupby(['date', 'store', 'product'])['num_sold'].mean().reset_index()

# Merge Norway averages with missing entries
missing_mask = train_df['num_sold'].isna()
missing_entries = train_df[missing_mask].drop(columns=['num_sold'])
missing_entries = missing_entries.merge(norway_avg, on=['date', 'store', 'product'], how='left', suffixes=('', '_norway'))

# Adjust num_sold using GDP ratios
missing_entries = missing_entries.merge(
    gdp_filtered.rename(columns={'country': 'country_norway', 'gdp_ratio': 'gdp_ratio_norway'}),
    left_on=['year'],
    right_on=['year'],
    how='left'
)
missing_entries = missing_entries[missing_entries['country_norway'] == 'Norway']

missing_entries['num_sold_imputed'] = missing_entries['num_sold'] * (missing_entries['gdp_ratio'] / missing_entries['gdp_ratio_norway'])

# Update original train_df
train_df.loc[missing_mask, 'num_sold'] = missing_entries['num_sold_imputed'].values

print(f"Missing values after imputation: {train_df['num_sold'].isna().sum()}")

# Handle any remaining missing values manually (if any)
remaining_missing = train_df[train_df['num_sold'].isna()]
if not remaining_missing.empty:
    # Assign specific values if necessary
    train_df.loc[train_df["id"] == 23719, "num_sold"] = 4
    train_df.loc[train_df["id"] == 207003, "num_sold"] = 195
    print(f"Missing values after manual assignment: {train_df['num_sold'].isna().sum()}")

# =========================================
# 4. CALCULATE STORE WEIGHTS
# =========================================
store_weights = train_df.groupby("store")["num_sold"].sum() / train_df["num_sold"].sum()
store_weights = store_weights.reset_index().rename(columns={"num_sold": "store_ratio"})

# =========================================
# 5. CALCULATE PRODUCT RATIOS
# =========================================
# Calculate average product ratios over all years
product_df = train_df.groupby(["date", "product"])["num_sold"].sum().reset_index()
product_pivot = product_df.pivot(index='date', columns='product', values='num_sold')
product_ratio = product_pivot.div(product_pivot.sum(axis=1), axis=0)
product_ratio = product_ratio.stack().reset_index().rename(columns={0: "product_ratio"})

# Instead of shifting previous years, use average ratios per (month, day)
product_ratio['month'] = product_ratio['date'].dt.month
product_ratio['day'] = product_ratio['date'].dt.day
avg_product_ratio = product_ratio.groupby(['month', 'day', 'product'])['product_ratio'].mean().reset_index()

# =========================================
# 6. AGGREGATE TIME SERIES
# =========================================
# Aggregate total sales per date
total_sales = train_df.groupby("date")["num_sold"].sum().reset_index()
total_sales["year"] = total_sales["date"].dt.year
total_sales["month"] = total_sales["date"].dt.month
total_sales["day"] = total_sales["date"].dt.day
total_sales["day_of_week"] = total_sales["date"].dt.dayofweek

# =========================================
# 7. ADJUST FOR DAY OF WEEK EFFECTS
# =========================================
# Calculate average num_sold per day of week
day_of_week_avg = total_sales.groupby("day_of_week")['num_sold'].mean().reset_index()
overall_avg = total_sales['num_sold'].mean()
day_of_week_avg['day_of_week_ratio'] = day_of_week_avg['num_sold'] / overall_avg
total_sales = total_sales.merge(day_of_week_avg[['day_of_week', 'day_of_week_ratio']], on='day_of_week', how='left')
total_sales['adjusted_num_sold'] = total_sales['num_sold'] / total_sales['day_of_week_ratio']

# =========================================
# 8. MAKE FORECAST
# =========================================
# Calculate average adjusted_num_sold per (month, day)
avg_adjusted_sales = total_sales.groupby(['month', 'day'])['adjusted_num_sold'].mean().reset_index()

# Prepare test_total_sales
test_dates = test_df['date'].drop_duplicates()
test_total_sales = pd.DataFrame({'date': test_dates})
test_total_sales['month'] = test_total_sales['date'].dt.month
test_total_sales['day'] = test_total_sales['date'].dt.day
test_total_sales['day_of_week'] = test_total_sales['date'].dt.dayofweek

# Merge with average adjusted sales and day of week ratios
test_total_sales = test_total_sales.merge(avg_adjusted_sales, on=['month', 'day'], how='left')
test_total_sales = test_total_sales.merge(day_of_week_avg[['day_of_week', 'day_of_week_ratio']], on='day_of_week', how='left')

# Handle missing adjusted_num_sold (if any) by using overall average
test_total_sales['adjusted_num_sold'].fillna(avg_adjusted_sales['adjusted_num_sold'].mean(), inplace=True)

test_total_sales['day_num_sold'] = test_total_sales['adjusted_num_sold'] * test_total_sales['day_of_week_ratio']

# =========================================
# 9. DISAGGREGATE TOTAL SALES FORECAST
# =========================================
# Merge test_df with test_total_sales
test_sub_df = test_df.merge(test_total_sales[['date', 'day_num_sold']], on='date', how='left')

# Add store ratios
test_sub_df = test_sub_df.merge(store_weights, on='store', how='left')

# Add country ratios
gdp_latest_year = gdp_filtered[gdp_filtered['year'] == gdp_filtered['year'].max()]
test_sub_df = test_sub_df.merge(gdp_latest_year[['country', 'gdp_ratio']], on='country', how='left')

# Add product ratios
test_sub_df['month'] = test_sub_df['date'].dt.month
test_sub_df['day'] = test_sub_df['date'].dt.day
test_sub_df = test_sub_df.merge(avg_product_ratio, on=['month', 'day', 'product'], how='left')

# Handle missing product_ratio (if any) by using overall average
overall_product_ratio = avg_product_ratio.groupby('product')['product_ratio'].mean().reset_index()
test_sub_df = test_sub_df.merge(overall_product_ratio, on='product', how='left', suffixes=('', '_overall'))
test_sub_df['product_ratio'].fillna(test_sub_df['product_ratio_overall'], inplace=True)

# Disaggregate to get num_sold
test_sub_df["num_sold"] = test_sub_df["day_num_sold"] * test_sub_df["store_ratio"] * test_sub_df["gdp_ratio"] * test_sub_df["product_ratio"]
test_sub_df["num_sold"] = test_sub_df["num_sold"].round()

# Ensure predictions are non-negative
test_sub_df["num_sold"] = test_sub_df["num_sold"].clip(lower=0)

# =========================================
# 10. SUBMISSION GENERATION
# =========================================
submission = pd.read_csv(SUBMISSION_PATH)
submission['num_sold'] = test_sub_df['num_sold'].values
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
submission.to_csv(f"sub_m13_{timestamp}.csv", index=False)
print("Submission saved!")
"""

In [10]:
import os
import json
import traceback

# ---------------------- LangChain or similar imports ----------------------
# Adjust as needed for your environment
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# --------------------------------------------------------------------
# 1. Setup your environment
# --------------------------------------------------------------------

# NEW: define the metric we want to use for evaluation
metric = "Mean Absolute Percentage Error (MAPE)"
mape_threshold = 0.05  # If MAPE is above this, we continue repairs

model_params = {
    "model": BASE_LLM,
    "api_key": OPENAI_API_KEY,
    "temperature": TEMPERATURE,
    "max_tokens": MAX_TOKENS
}
llm = ChatOpenAI(**model_params)

# --------------------------------------------------------------------
# 2. Prepare data context
# --------------------------------------------------------------------
train_data_path = "/kaggle/input/playground-series-s5e1/train.csv"
test_data_path = "/kaggle/input/playground-series-s5e1/test.csv"
submission_example_path = "/kaggle/input/playground-series-s5e1/sample_submission.csv"
gdp_path = "/kaggle/input/world-gdpgdp-gdp-per-capita-and-annual-growths/gdp_ppp_per_capita.csv"
submission_path = "/kaggle/working/submission.csv"

train_data_summary = context
train_data_summary_json = json.dumps(train_data_summary, indent=2)

target_variable = "num_sold"

# --------------------------------------------------------------------
# 3. Prepare your initial prompt template
# --------------------------------------------------------------------
prompt_template = """
You are given the following dataset information:
- Train data path: {train_path}
- GDP information for feature enginering path: {gdp_path}
- Test data path: {test_path}
- Submission example path: {submission_example_path}
- Train data summary: {train_summary}
- Target variable: {target_var}
- Path to the final submission file to generate: {submission_path}

**Task**:
Generate a complete Python script that does the following:
1. Reads the train data from the provided path (EXACTLY use the variable {train_path}).
2. Creates any helpful new features based on the data summary.
3. Trains a model to predict the target variable, which is {target_var}.
4. Uses the test data (from {test_path}) to create predictions.
5. Generates a submission file in the path {submission_path}, 
   using {submission_example_path} if needed for reference.
6. Print or log any relevant steps (like shape of data, MAPE, etc.) so we can see progress.
7. Handle missing values in the target variable if necessary.
8. Return only valid Python code, with no triple backticks or markdown fences.

Important:
- Do **not** use placeholders like 'train.csv' by itself. You must use the actual path: {train_path}, {test_path}, etc.
- The script must be fully self-contained: from import statements to reading/writing files.
- Do **not** wrap the code in ```python or similar.

Additionally, please compute the '{metric}' during validation (or after training) and store it in a variable (e.g., val_mape). 
We'll be using '{metric}' to evaluate the model. 
If your MAPE is above 0.10, we will attempt repairs or further improvements.

see below my best code template so far:

{best_model_script}

Begin now.
"""

chain_prompt = ChatPromptTemplate.from_template(template=prompt_template)
initial_chain = chain_prompt | llm | StrOutputParser()

# --------------------------------------------------------------------
# 4. Prepare a "repair" prompt template
# --------------------------------------------------------------------
repair_prompt_template = """
The following code was executed but caused an error or had an unsatisfactory {metric} above 0.10. 
Here is the **entire code** that was run:


Here is the **traceback** (if any):


**Task**: 
1. Provide a **complete** corrected Python script that still fulfills all the original requirements:
   - Use the actual paths provided: {train_path}, {test_path}, {submission_example_path}, {submission_path}.
   - Create new features, train a model to predict {target_var}, and save predictions to {submission_path}.
   - Print relevant info (shapes, validation score, {metric}, etc.) so we can track progress.
2. If there are missing values in the target variable, show how you handle them.
3. **Do not** enclose the script in triple backticks or any markdown fences.
4. The script should be fully self-contained (all imports, reading data, writing submission, etc.).
5. Please ensure the final code variable storing {metric} is named val_mape (i.e., val_mape = ...).

Begin now.
"""

repair_chain_prompt = ChatPromptTemplate.from_template(template=repair_prompt_template)
repair_chain = repair_chain_prompt | llm | StrOutputParser()

# --------------------------------------------------------------------
# 5. Utility function to clean out any leftover backticks (just in case)
# --------------------------------------------------------------------
def remove_markdown_code_fences(code_str: str) -> str:
    """
    Remove triple-backtick fences from code.
    Also removes lines that start with ``` or contain them.
    """
    lines = code_str.splitlines()
    cleaned = []
    for line in lines:
        if "```" not in line:
            cleaned.append(line)
    return "\n".join(cleaned).strip()

# --------------------------------------------------------------------
# 6. Iterative generation + repair loop
# --------------------------------------------------------------------
current_iteration = 0
error_encountered = True
repaired_code = None

while current_iteration < MAX_ITERATIONS and error_encountered:
    current_iteration += 1
    
    if current_iteration == 1:
        # Step 1: Use the initial chain to generate code
        result_code = initial_chain.invoke({
            "train_path": train_data_path,
            "gdp_path": gdp_path,
            "test_path": test_data_path,
            "submission_example_path": submission_example_path,
            "train_summary": train_data_summary_json,
            "target_var": target_variable,
            "submission_path": submission_path,
            "metric": metric,
            "best_model_script" : best_model_script
        })
    else:
        # If we already tried once, we use the "repaired_code" from the LLM
        result_code = repaired_code

    # Clean out triple backticks just in case
    cleaned_code = remove_markdown_code_fences(result_code)
    
    print(f"\n--- Attempt #{current_iteration}: Cleaned Code ---\n")
    print(cleaned_code)
    print("\n--- End of Cleaned Code ---\n")
    
    try:
        # Execute code in a local namespace so we can retrieve variables (like val_mape)
        local_ns = {}
        exec(cleaned_code, {}, local_ns)
        print(f"Code executed successfully on attempt #{current_iteration}!\n")

        # If the generated script provides "val_mape" in local_ns, let's check it:
        if "val_mape" in local_ns:
            val_mape = local_ns["val_mape"]
            print(f"Retrieved MAPE from script: {val_mape}")
            
            # If MAPE is above threshold, treat it as if an "error" occurred
            if val_mape > mape_threshold:
                raise ValueError(
                    f"MAPE {val_mape:.4f} is above threshold {mape_threshold}. "
                    "Triggering a repair attempt."
                )
        
        # If we get here, either there's no val_mape or it's below threshold
        error_encountered = False

    except Exception as e:
        error_encountered = True
        error_trace = traceback.format_exc()
        print(f"Error on attempt #{current_iteration}:\n{error_trace}")
        
        if current_iteration < MAX_ITERATIONS:
            # Step 2: Feed the failing code + traceback to the repair chain
            repaired_code = repair_chain.invoke({
                "generated_code": cleaned_code,
                "error_trace": error_trace,
                "train_path": train_data_path,
                "test_path": test_data_path,
                "submission_example_path": submission_example_path,
                "submission_path": submission_path,
                "target_var": target_variable,
                "metric": metric
            })
        else:
            print("Max iterations reached. Still failing or MAPE still too high. Exiting loop.\n")
            break

print("Process complete.")


--- Attempt #1: Cleaned Code ---

import numpy as np
import pandas as pd
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings("ignore", category=FutureWarning)

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Paths to the datasets
TRAIN_PATH = "/kaggle/input/playground-series-s5e1/train.csv"
TEST_PATH = "/kaggle/input/playground-series-s5e1/test.csv"
GDP_PATH = "/kaggle/input/world-gdpgdp-gdp-per-capita-and-annual-growths/gdp_ppp_per_capita.csv"
SUBMISSION_PATH = "/kaggle/input/playground-series-s5e1/sample_submission.csv"
FINAL_SUBMISSION_PATH = "/kaggle/working/submission.csv"

# Load datasets
train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
test_df = pd.read_csv(TEST_PATH, parse_dates=["date"])
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)